In [2]:
# import the libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
import warnings
warnings.filterwarnings('ignore', category= UserWarning)
print("Complete")

C:\Users\INSARAJ22\AppData\Local\anaconda3\envs\personal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Complete


In [15]:
# Read the data
file='train'
data = pd.read_csv(rf'{file}.csv')
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [16]:
# Check the data-set info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [17]:
# Check for missing values
(data.isnull().sum() / len(data)) * 100

Item_Identifier               0.000000
Item_Weight                  17.165317
Item_Fat_Content              0.000000
Item_Visibility               0.000000
Item_Type                     0.000000
Item_MRP                      0.000000
Outlet_Identifier             0.000000
Outlet_Establishment_Year     0.000000
Outlet_Size                  28.276428
Outlet_Location_Type          0.000000
Outlet_Type                   0.000000
Item_Outlet_Sales             0.000000
dtype: float64

###### **Only two cols has missing items Item_Weight and Outlet_Size**
###### **Item_Weight has 17% missing values and Outlet_Size has 28% missing values**

In [18]:
# Identify the categorical cols in the data
categorical_cols = ['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 
                    'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
numerical_cols = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales', 'quantiyty']
print(f"Categoral Cols: {categorical_cols}")
print(f"Numerical Cols: {numerical_cols}")

Categoral Cols: ['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
Numerical Cols: ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales', 'quantiyty']


In [19]:
(data["Item_Fat_Content"].value_counts() / len(data["Item_Fat_Content"])) * 100

# remove the inconsistencies
data["Item_Fat_Content"] = data["Item_Fat_Content"].replace(
    {"LF": "Low Fat",
    "low fat": "Low Fat",
    "reg": "Regular"}
)

In [20]:
print(data["Item_Fat_Content"].unique())

['Low Fat' 'Regular']


###### **ItemFatContent has two anomalies LF and reg they should be Low Fat and Regular**

In [21]:
# # Check the correlation among the features
# plt.figure(figsize= (16, 10))
# sns.heatmap(data[numerical_cols].corr(), annot= True)
# plt.show()

###### **Outlet Sales have high correlation with Item_MRP**

In [22]:
# Treat the missing values
# The distribution of the Item_Weight is uniform (so Mean ~ Median ~ Mode) and no outliers
# Same items should have the same weight
data["Item_Weight"] = data.groupby("Item_Identifier")["Item_Weight"].transform(lambda x: x.fillna(x.median()))
data["Outlet_Size"] = data.groupby("Outlet_Type")["Outlet_Size"].transform(lambda x: x.fillna(x.mode()[0]))
# Still some values are missing so impute them using
data["Item_Weight"] = data.groupby(["Item_Fat_Content", "Item_Type"])["Item_Weight"].transform(lambda x: x.fillna(x.median()))

C:\Users\INSARAJ22\AppData\Local\anaconda3\envs\personal\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\INSARAJ22\AppData\Local\anaconda3\envs\personal\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\INSARAJ22\AppData\Local\anaconda3\envs\personal\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\INSARAJ22\AppData\Local\anaconda3\envs\personal\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [23]:
def transform_data(df: pd.DataFrame):
    """
    Encodes categorical variables in the dataset as per the encoding strategy.
    Ensures a robust transformation for production-level implementation.
    """
    df["Outlet_Age"] = 2013 - df["Outlet_Establishment_Year"]
    df["Price_Per_Weight"] = df["Item_MRP"] / df["Item_Weight"]
    df.drop("Outlet_Age", axis = 1, inplace = True)
    #  Feature Extraction from Item_Identifier --------------------
    df["Item_Category"] = df["Item_Type"].replace({
    "Dairy": "Perishable",
    "Meat": "Perishable",
    "Fruits and Vegetables": "Perishable",
    "Baking Goods": "Processed",
    "Breakfast": "Processed",
    "Canned": "Processed",
    "Frozen Foods": "Processed",
    "Hard Drinks": "Drinks",
    "Soft Drinks": "Drinks",
    "Health and Hygiene": "Non-Food",
    "Household": "Non-Food",
    "Others": "Non-Food",
    "Seafood": "Perishable",
    "Snack Foods": "Processed",
    "Starchy Foods": "Processed"
    })
    df.drop(["Item_Identifier"], axis= 1, inplace = True) # drop the cols 

    # Label Encoding --------------------
    label_encoders = {}

    # Label encode Outlet_Identifier (as it represents unique stores)
    label_encoders["Outlet_Identifier"] = LabelEncoder()
    df["Outlet_Identifier"] = label_encoders["Outlet_Identifier"].fit_transform(df["Outlet_Identifier"])

    # Label encode Outlet_Size (Small=0, Medium=1, Large=2)
    size_mapping = {"Small": 0, "Medium": 1, "High": 2}
    df["Outlet_Size"] = df["Outlet_Size"].map(size_mapping)

    # Label encode Item_Fat_Content (Low Fat = 1, Regular = 0)
    fat_mapping = {"Low Fat": 1, "Regular": 0}
    df["Item_Fat_Content"] = df["Item_Fat_Content"].replace(fat_mapping)

    # One-Hot Encoding --------------------
    one_hot_cols = ["Item_Category", "Outlet_Location_Type", "Outlet_Type"]
    df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True)  # Drop first to avoid multicollinearity

    # Also one-hot encode the extracted Item_Category_Code
    df = pd.get_dummies(df, columns=["Item_Category_Code"], drop_first=True)

    # Final Check & Return --------------------
    return df


# Example Usage
# df_encoded = encode_data(df)
# print(df_encoded.head())


# Fix the 0 Item_Visibilities
def fix_item_visiblity(data,item_visibility_median,item_weight_medians,outlet_size_modes,outlet_size_modes_2,item_weight_medians_2):
    """Replace zero values in Item_Visibility using median visibility of that item"""
    median_visibilty_per_item = data.groupby("Item_Identifier")["Item_Visibility"].median()
    data.loc[data["Item_Visibility"] == 0, "Item_Visibility"] = data["Item_Identifier"].map(median_visibilty_per_item)
    
    # If NaN still exist replace with overall median
    data["Item_Visibility"].fillna(item_visibility_median, inplace= True)

    # Step 2: Fill missing values in train data
#     data["Item_Weight"] = data["Item_Weight"].fillna(data["Item_Identifier"].map(item_weight_medians))
#     data["Outlet_Size"] = data["Outlet_Size"].fillna(data["Outlet_Type"].map(outlet_size_modes))
#     data["Item_Weight"] = data["Item_Weight"].fillna(data[["Item_Fat_Content", "Item_Type"]].apply(lambda x: item_weight_medians_2.get(tuple(x)), axis=1))
#     data["Outlet_Size"] = data["Outlet_Size"].fillna(data[["Outlet_Type", "Outlet_Location_Type"]].apply(lambda x: outlet_size_modes_2.get(tuple(x)), axis=1))


    data["Item_Weight"] = data.groupby("Item_Identifier")["Item_Weight"].transform(lambda x: x.fillna(x.median()))
    data["Outlet_Size"] = data.groupby("Outlet_Type")["Outlet_Size"].transform(lambda x: x.fillna(x.mode()[0]))
    # Still some values are missing so impute them using
    data["Outlet_Size"] = data.groupby(["Outlet_Type", "Outlet_Location_Type"])["Outlet_Size"].transform(lambda x: x.fillna(x.mode()[0]))
    data["Item_Weight"] = data.groupby(["Item_Fat_Content", "Item_Type"])["Item_Weight"].transform(lambda x: x.fillna(x.median()))
    
    data["Item_Fat_Content"] = data["Item_Fat_Content"].replace(
    {"LF": "Low Fat",
    "low fat": "Low Fat",
    "reg": "Regular"}
    )
    
    return data

In [24]:
def optimize_hyperparams(X, y, n_trials=50):
    """
    Optimizes hyperparameters for XGBoost and Random Forest using Optuna.
    Returns the best trained model and its parameters.
    """
    def objective(trial):
        model_type = trial.suggest_categorical("model_type", ["xgboost", "random_forest"])
        
        if model_type == "xgboost":
            params = {
                "n_estimators": trial.suggest_categorical("n_estimators", [1,2,3,4,5,7,10,20,50,75,100,200,500,700]),
                "max_depth": trial.suggest_int("max_depth", 2,30),
                "learning_rate": trial.suggest_categorical("learning_rate", [0.0001,0.001,0.005,0.01,0.05,0.1,0.3,0.5]),
#                 "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "gamma":trial.suggest_categorical("gamma", [0.01,0.05,0.1,0.3,0.5,1,5,10]),
                "min_child_weight":trial.suggest_categorical("min_child_weight", [0.01,0.05,0.1,0.3,0.5,1,5,10]),
                "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.2,0.5,0.7,1.0]),
#                 "reg_alpha": trial.suggest_categorical("reg_alpha", [0.01,0.05,0.1,0.3,0.5,1,5,10]),
                "reg_lambda": trial.suggest_categorical("reg_lambda", [0.01,0.05,0.1,0.3,0.5,1,5,10])
            }
            model = xgb.XGBRegressor(**params, objective="reg:squarederror", random_state=101)
        
        else:  # Random Forest
            params = {
                "n_estimators": trial.suggest_categorical("n_estimators", [1,2,3,4,5,7,10,20,50,75,100,200,500,700]),
                "max_depth": trial.suggest_int("max_depth", 2,30),
                "min_samples_split": trial.suggest_categorical("min_samples_split", [2,3,4,5,7,10,15,20,30]),
                "min_samples_leaf": trial.suggest_categorical("min_samples_leaf", [2,3,4,5,7,10,15,20,30]),
                "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
            }
            model = RandomForestRegressor(**params, random_state=101, n_jobs=-1)
        
        # Perform cross-validation
        scores = cross_val_score(model, X, y, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1)
        return np.mean(scores)  # Maximize negative RMSE (minimizing RMSE)

    # Run Optuna optimization
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # Get the best model type and parameters
    best_params = study.best_params
    best_model_type = best_params.pop("model_type")

    # Train the final best model
    if best_model_type == "xgboost":
        best_model = xgb.XGBRegressor(**best_params, objective="reg:squarederror", random_state=42)
    else:
        best_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)

    # Fit the best model on the full dataset
    best_model.fit(X, y)

    return best_model, best_params, study, best_model_type


In [25]:
# Compute median values from the train data
if file=='train':
    item_weight_medians = data.groupby("Item_Identifier")["Item_Weight"].median()
    item_visibility_median=data['Item_Visibility'].median()
    # 1st Imputation - Using Outlet_Type
    outlet_size_modes = data.groupby("Outlet_Type")["Outlet_Size"].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

    # 2nd Imputation - Using Outlet_Type and Outlet_Location_Type
    
    outlet_size_modes_2 = data.groupby(["Outlet_Type", "Outlet_Location_Type"])["Outlet_Size"].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
    # 3rd Imputation - Using Item_Fat_Content and Item_Type
    item_weight_medians_2 = data.groupby(["Item_Fat_Content", "Item_Type"])["Item_Weight"].median()

processed_data = fix_item_visiblity(data.copy(),item_visibility_median,item_weight_medians,outlet_size_modes,outlet_size_modes_2,item_weight_medians_2)

In [26]:
df = processed_data.copy()

In [27]:
df["Outlet_Age"] = 2025 - df["Outlet_Establishment_Year"]
df["Price_Per_Weight"] = df["Item_MRP"] / df["Item_Weight"]
df["Item_Category_Code"] = df["Item_Identifier"].apply(lambda x: x[:2])

In [28]:
df["Item_Category"] = df["Item_Type"].replace({
"Dairy": "Perishable",
"Meat": "Perishable",
"Fruits and Vegetables": "Perishable",
"Baking Goods": "Processed",
"Breakfast": "Processed",
"Canned": "Processed",
"Frozen Foods": "Processed",
"Hard Drinks": "Drinks",
"Soft Drinks": "Drinks",
"Health and Hygiene": "Non-Food",
"Household": "Non-Food",
"Others": "Non-Food",
"Seafood": "Perishable",
"Snack Foods": "Processed",
"Starchy Foods": "Processed"
})

In [29]:
# Label Encoding --------------------
label_encoders = {}

# Label encode Outlet_Identifier (as it represents unique stores)
# label_encoders["Outlet_Identifier"] = LabelEncoder()
# df["Outlet_Identifier"] = label_encoders["Outlet_Identifier"].fit_transform(df["Outlet_Identifier"])

# Label encode Outlet_Size (Small=0, Medium=1, Large=2)
# size_mapping = {"Small": 0, "Medium": 1, "High": 2}
# df["Outlet_Size"] = df["Outlet_Size"].map(size_mapping)

# Label encode Item_Fat_Content (Low Fat = 1, Regular = 0)
fat_mapping = {"Tier 1": 2, "Tier 2": 1, "Tier 3": 0}
df["Outlet_Location_Type"] = df["Outlet_Location_Type"].replace(fat_mapping)

fat_mapping = {"Grocery Store": 0, "Supermarket Type3": 1, "Supermarket Type2":1, "Supermarket Type1":1}
df["Outlet_Type"] = df["Outlet_Type"].replace(fat_mapping)

fat_mapping = {"Low Fat": 0, "Regular": 1}
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace(fat_mapping)
one_hot_cols = ["Outlet_Type","Outlet_Identifier"]

In [30]:
df.to_csv(rf'{file}_transformed.csv',index=False)

In [31]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Age,Price_Per_Weight,Item_Category_Code,Item_Category
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Medium,2,1,3735.1380,26,26.861204,FD,Perishable
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,0,1,443.4228,16,8.153581,DR,Drinks
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,Medium,2,1,2097.2700,26,8.092457,FD,Perishable
3,FDX07,19.20,1,0.022861,Fruits and Vegetables,182.0950,OUT010,1998,Small,0,0,732.3800,27,9.484115,FD,Perishable
4,NCD19,8.93,0,0.006590,Household,53.8614,OUT013,1987,High,0,1,994.7052,38,6.031512,NC,Non-Food


In [32]:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'Outlet_Age', 'Price_Per_Weight',
       'Item_Category_Code', 'Item_Category'],
      dtype='object')

In [33]:
columns_to_drop=["Item_Type", "Item_Identifier", "Item_Category_Code", "Item_Category",
             "Outlet_Establishment_Year","Outlet_Location_Type","Outlet_Size"]

In [36]:
from sklearn.preprocessing import OneHotEncoder

def train(one_hot_cols):
    df=pd.read_csv('train_transformed.csv')
    df.drop(columns_to_drop, axis= 1, inplace = True) # drop the cols 

    # One-Hot Encoding
    encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoded = encoder.fit_transform(df[one_hot_cols])
    one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(one_hot_cols))
    df = pd.concat([df.drop(one_hot_cols, axis=1), one_hot_df], axis=1)
    
    #splitting
    X=df.drop('Item_Outlet_Sales',axis=1)
    y=df['Item_Outlet_Sales']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    #training
    best_model, best_params, study, best_model_type=optimize_hyperparams(X_train,y_train)
    y_pred=best_model.predict(X_test)
    rms = mean_squared_error(y_pred, y_test, squared=False)
    print(rms)
    
    fi=pd.DataFrame(df.drop('Item_Outlet_Sales',axis=1).columns,columns=['Columns'])
    fi['Feature importance']=best_model.feature_importances_*100
    
    #train the best model on full data
    if best_model_type == "xgboost":
        model = xgb.XGBRegressor(**best_params, objective="reg:squarederror", random_state=101, n_jobs=-1)
    else:
        model = RandomForestRegressor(**best_params, random_state=101, n_jobs=-1)
    model.fit(X=df.drop('Item_Outlet_Sales',axis=1),y=df['Item_Outlet_Sales'])
    
    return model, best_params, study, rms, encoder, fi

In [37]:
#execute only for training
best_model, best_params, study, rms, encoder, fi=train(one_hot_cols)
fi

[I 2025-02-01 03:57:16,513] A new study created in memory with name: no-name-2a646e4d-f8a1-4a0a-8c6a-df8bf0abad83
[W 2025-02-01 03:57:16,518] Trial 0 failed with parameters: {'model_type': 'xgboost', 'n_estimators': 20, 'max_depth': 12, 'learning_rate': 0.0001, 'gamma': 0.5, 'min_child_weight': 5, 'colsample_bytree': 1.0, 'reg_lambda': 0.05} because of the following error: AttributeError("'super' object has no attribute '__sklearn_tags__'").
Traceback (most recent call last):
  File "C:\Users\INSARAJ22\AppData\Local\anaconda3\envs\personal\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\INSARAJ22\AppData\Local\Temp\ipykernel_19032\3294885818.py", line 34, in objective
    scores = cross_val_score(model, X, y, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\INS

AttributeError: 'super' object has no attribute '__sklearn_tags__'

,Columns,Feature importance
0,Item_Weight,3.585264
1,Item_Fat_Content,0.316948
2,Item_Visibility,2.465384
3,Item_MRP,36.974926
4,Outlet_Age,5.281721
5,Price_Per_Weight,18.193258
6,Outlet_Type_0,9.853572
7,Outlet_Type_1,10.935565
8,Outlet_Identifier_OUT010,2.769981
9,Outlet_Identifier_OUT013,0.124038


In [ ]:
best_params

In [92]:
#generate test results
df_test=pd.read_csv('test_transformed.csv')
df_test.drop(columns_to_drop, axis= 1, inplace = True)

# One-Hot Encoding --------------------
one_hot_encoded = encoder.transform(df_test[one_hot_cols])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(one_hot_cols))
df_test = pd.concat([df_test.drop(one_hot_cols, axis=1), one_hot_df], axis=1)

pred=best_model.predict(df_test)
df_test=pd.read_csv('test_transformed.csv')
df_submit=df_test[['Item_Identifier','Outlet_Identifier']]
df_submit['Item_Outlet_Sales']=pred
df_submit.to_csv('submission.csv',index=False)

C:\Users\INSARAJ22\AppData\Local\Temp\ipykernel_15104\1239011212.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submit['Item_Outlet_Sales']=pred
